In [1]:
using JuMP
using Gurobi

In [2]:
time_=8760
F=[0.03*2000,0.03*950,0.03*700]
EA=[175.2252157,85.60445144,61.91153839]
C=[1,1,1]
l=5000
H=[2.4,1.62,2.5]
P=[5,30,30]

3-element Vector{Int64}:
  5
 30
 30

In [3]:
using CSV
using DataFrames

df = CSV.read("data.csv", DataFrame)
D_t = df[!, "FR_load_forecast_entsoe_transparency"]
println(size(D_t))

(8760,)


In [4]:
model=Model(Gurobi.Optimizer)
@variable(model,0<=p[1:3,1:time_])
@variable(model,k[1:3])
@variable(model,0<=d[1:time_])
@objective(model,Min,sum((F[g]+EA[g])*k[g]+sum(H[g]*P[g]*p[g,i]/1000 for i in 1:time_) for g in 1:3)-sum(l*d[i] for i in 1:time_))
@constraint(model,[g in 1:3,i in 1:time_],p[g,i] <= C[g,1]*k[g])
@constraint(model,[i in 1:time_],d[i] <= D_t[i])
@constraint(model,[i in 1:time_],d[i]-sum(p[g,i] for g in 1:3)==0)

Set parameter Username
Academic license - for non-commercial use only - expires 2024-10-31


8760-element Vector{ConstraintRef{Model, MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64}, MathOptInterface.EqualTo{Float64}}, ScalarShape}}:
 -p[1,1] - p[2,1] - p[3,1] + d[1] == 0
 -p[1,2] - p[2,2] - p[3,2] + d[2] == 0
 -p[1,3] - p[2,3] - p[3,3] + d[3] == 0
 -p[1,4] - p[2,4] - p[3,4] + d[4] == 0
 -p[1,5] - p[2,5] - p[3,5] + d[5] == 0
 -p[1,6] - p[2,6] - p[3,6] + d[6] == 0
 -p[1,7] - p[2,7] - p[3,7] + d[7] == 0
 -p[1,8] - p[2,8] - p[3,8] + d[8] == 0
 -p[1,9] - p[2,9] - p[3,9] + d[9] == 0
 -p[1,10] - p[2,10] - p[3,10] + d[10] == 0
 ⋮
 -p[1,8752] - p[2,8752] - p[3,8752] + d[8752] == 0
 -p[1,8753] - p[2,8753] - p[3,8753] + d[8753] == 0
 -p[1,8754] - p[2,8754] - p[3,8754] + d[8754] == 0
 -p[1,8755] - p[2,8755] - p[3,8755] + d[8755] == 0
 -p[1,8756] - p[2,8756] - p[3,8756] + d[8756] == 0
 -p[1,8757] - p[2,8757] - p[3,8757] + d[8757] == 0
 -p[1,8758] - p[2,8758] - p[3,8758] + d[8758] == 0
 -p[1,8759] - p[2,8759] - p[3,8759] + d[8759] == 0
 -p[1,8760] - p[2,8760

Question (i) : optimal installed capacity mix

In [5]:
optimize!(model)
println("Variables value:",JuMP.value.(k))
println("Objective value: ", objective_value(model))

Gurobi Optimizer version 10.0.3 build v10.0.3rc0 (win64)

CPU model: Intel(R) Core(TM) i7-10510U CPU @ 1.80GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 43800 rows, 35043 columns and 96360 nonzeros
Model fingerprint: 0xe24638f8
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e-02, 5e+03]
  Bounds range     [0e+00, 0e+00]
  RHS range        [3e+04, 1e+05]
Presolve removed 8760 rows and 8760 columns
Presolve time: 0.06s
Presolved: 35040 rows, 26283 columns, 78840 nonzeros

Concurrent LP optimizer: dual simplex and barrier
Showing barrier log only...

Ordering time: 0.00s

Barrier statistics:
 Dense cols : 3
 AA' NZ     : 6.834e+03
 Factor NZ  : 2.506e+04 (roughly 3 MB of memory)
 Factor Ops : 1.435e+05 (less than 1 second per iteration)
 Threads    : 1

                  Objective                Residual
Iter       Primal          Dual         Primal    Dual     Compl 

Question (ii) : share of each technology in the effective production

In [6]:
p1=sum(JuMP.value.(p[1,t]) for t in 1:8760)
p2=sum(JuMP.value.(p[2,t]) for t in 1:8760)
p3=sum(JuMP.value.(p[3,t]) for t in 1:8760)
total=sum(JuMP.value.(d[t]) for t in 1:8760)
println("ptot : ",total)
println("p1 : ",p1)
println("p2 : ",p2)
println("p3 : ",p3)
println("share of p1 : ",p1/total)
println("share of p2 : ",p2/total)
println("share of p3 : ",p3/total)

ptot : 4.72035818e8
p1 : 4.3547595e8
p2 : 2.92589925e7
p3 : 7.3008755e6
share of p1 : 0.9225485299931201
share of p2 : 0.06198468714507593
share of p3 : 0.015466782861803932


Question (iii) : total production and investment cost

In [7]:
cost=sum((EA[g]+F[g])*JuMP.value.(k[g])+sum(H[g]*P[g]/1000*JuMP.value.(p[g,i]) for i in 1:time_)  for g in 1:3)

2.3995146621966127e7

Question (iv) : total CO2 emitted by your system

In [8]:
CO2=[1.4,0.5,0.6]
production=[p1,p2,p3]
emissions= sum(CO2[t]*production[t,1] for t in 1:3)

6.2867635155e8

Question (v) :  amount of load curtailment

In [9]:
sum(max(0, D_t[t] - sum(JuMP.value(p[g, t]) for g in 1:3)) for t in 1:time_)

0.0

Question (vi) : total profit of each technology

In [10]:
profit=zeros(3,1)
for g in 1:3
    profit[g,1] = -(EA[g]+F[g])*JuMP.value.(k[g])-sum((H[g]*P[g]-l)/1000*JuMP.value.(p[g,i]) for i in 1:time_) 
end
println(profit)

[2.1590284715640225e9; 1.4336679670549178e8; 3.3788675108601496e7;;]


LCOE

In [11]:
LCOE=zeros(3,1)
for g in 1:3
    LCOE[g,1]=(((EA[g]+F[g])*JuMP.value.(k[g]))+(sum((H[g]*P[g])/1000*JuMP.value.(p[g,i]) for i in 1:time_)))*1000/production[g]
end
println(LCOE)

[42.14073919824874; 100.07746488564223; 371.96941536648586;;]
